# Spaced Repetition Algorithim

## Ebbinghaus
The ebbinghaus model will represent human memory decay.

\begin{equation}
p=2^{-\frac{\Delta}{h}}\\
\Delta = \text{time since last review}\\
h = \text{half life (memory decay constant)}\\
p = \text{probability of recall}\\
\end{equation}

Intially for our purposes, we will assume that the ideal time to schedule a review is at $p=0.5$.
This part of the algorithim is simply the half life approximation. Scheduling will come into play later, allow different thresholds of p, and adapting to different $\Delta$ that the user will likely control.

## Finding Half Life

For each existing algorithim, we must conver them to a generalized ebbinghaus model, to be able to compare them to historical data.

\begin{equation}
h = -\frac{\Delta}{\log_2 p_a}
\end{equation}



### Leitner
Here we outline leitner in a different way. For any given card, when should I review it again?
In other words when:

\begin{equation}
0.5=2^{-\frac{\Delta}{h}}\\
h = x_{\oplus} - x_{\ominus} \quad \text{(Num correct and Num wrong)}
\end{equation}


### SM-2
EF = min(1/125 (-100 + 7. q_v - 0.1 q_v^2) + 2.5, 1.3)
 x EF
\begin{equation}
0.5=2^{-\frac{\Delta}{h}}\\
h = x_{\oplus} - x_{\ominus} \quad \text{(Num correct and Num wrong)}
\end{equation}

## Evaluation
To evaluate each algorithim, we will be using historical data to try and estimate a theoretical half life. However most algorithims always reccommend delta, or the interval at which the next review session should be held. The paper makes an assumption that all algorithims must initiate a review session at $p=0.5$. In this evaluation, we will compare performance across multiple values of $p$.\
\
$\mathcal{D} = \{\langle p, \Delta, \mathbf{x}\rangle\}^D_i \quad$ This represents the dataset we have\
\
We will calculate a loss function using the observed recall and a theoretical recall. The theoretical recall is computed via $\Delta$ and $h_\Theta$. $h_\Theta$ is a function that takes a set of input features and estimates a half life. 

In [2]:
import numpy as np
import pandas as pd
import torch

In [3]:
df = pd.read_csv('./data/settles.acl16.learning_traces.13m.csv')

df

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,lexeme_string,history_seen,history_correct,session_seen,session_correct
0,1.000000,1362076081,27649635,u:FO,de,en,76390c1350a8dac31186187e2fe1e178,lernt/lernen<vblex><pri><p3><sg>,6,4,2,2
1,0.500000,1362076081,27649635,u:FO,de,en,7dfd7086f3671685e2cf1c1da72796d7,die/die<det><def><f><sg><nom>,4,4,2,1
2,1.000000,1362076081,27649635,u:FO,de,en,35a54c25a2cda8127343f6a82e6f6b7d,mann/mann<n><m><sg><nom>,5,4,1,1
3,0.500000,1362076081,27649635,u:FO,de,en,0cf63ffe3dda158bc3dbd55682b355ae,frau/frau<n><f><sg><nom>,6,5,2,1
4,1.000000,1362076081,27649635,u:FO,de,en,84920990d78044db53c1b012f5bf9ab5,das/das<det><def><nt><sg><nom>,4,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
12854221,0.800000,1363104897,368,u:i5D8,en,it,d5efc552aaea3109eb5388aa1ec8673d,the/the<det><def><sp>,6,4,5,4
12854222,0.800000,1363104897,368,u:i5D8,en,it,a826c47947d68549fa81e19cafa57ba0,eat/eat<vblex><pres>,4,4,5,4
12854223,1.000000,1363104897,368,u:i5D8,en,it,5e29d77697d23070a1fb92eb6c90e9b6,bread/bread<n><sg>,4,4,4,4
12854224,0.600000,1363104897,368,u:i5D8,en,it,cdfecc9247566d40bb964a218c54c783,drink/drink<vblex><pres>,3,2,5,3


In [24]:
df_sub = df[["p_recall", "timestamp","delta", "user_id", "lexeme_id", "history_seen", "history_correct", "session_seen", "session_correct"]]

In [25]:
raw_data = df_sub.values

In [27]:
print(raw_data)

[[1.0 1362076081 27649635 ... 4 2 2]
 [0.5 1362076081 27649635 ... 4 2 1]
 [1.0 1362076081 27649635 ... 4 1 1]
 ...
 [1.0 1363104897 368 ... 4 4 4]
 [0.6 1363104897 368 ... 2 5 3]
 [0.666666666667 1363104897 368 ... 3 9 6]]


In [26]:
user_item_history = {}

for i, row in enumerate(raw_data):
    user_id = row[3]
    item_id = row[4]
    
    if user_id not in user_item_history:
        user_item_history[user_id] = {}
        
    if item_id not in user_item_history[user_id]:
        user_item_history[user_id][item_id] = []
        
    user_item_history[user_id][item_id].append(i)



In [28]:
user_item_history['u:FO']

{'76390c1350a8dac31186187e2fe1e178': [0, 8, 51, 57, 59],
 '7dfd7086f3671685e2cf1c1da72796d7': [1, 9, 52],
 '35a54c25a2cda8127343f6a82e6f6b7d': [2, 10, 17, 61],
 '0cf63ffe3dda158bc3dbd55682b355ae': [3, 11, 55],
 '84920990d78044db53c1b012f5bf9ab5': [4, 12, 16, 63],
 '56429751fdaedb6e491f4795c770f5a4': [5, 13, 62],
 '1bacf218eaaf9f944e525f7be9b31899': [6, 14, 60],
 '4fcb6bb8e44d7b618999721071862827': [18],
 'a6834806c43ea1be9eb3e4fdae6f98db': [19],
 'dd34978165d17f7e729a2ef331a7600d': [53, 58],
 '495f763ef6027e020c53431484aa5ede': [54],
 '46b112cd07fdcc98db8670a5d71c613d': [56]}